In [1]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [3]:
import re
import string
import unicodedata
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
import pickle

def clean_text(kata):    
    # remove full urls
    kata = re.sub("https*\S+", "<URL>", kata)
    
    #lowercase
    kata = kata.lower()
    
    # remove full hastag
    # kata = re.sub("#\S+", " ", kata)
    
    #remove number
    # kata = re.sub(r"\d+", "", kata)
    
    # remove link
    # kata = re.sub(r"http\S+", "", kata)
    
    # remove hashtag
    # kata = kata.replace('#', '')
    
    # remove space >, <
    kata = kata.replace('< ', '<')
    kata = kata.replace(' >', '>')
    
    # remove punctuation
    # kata = kata.translate(str.maketrans("","",string.punctuation))
    
    #remove extra spaces
    kata = kata.strip()
    
    # remove non ascii chars
    new_words = []
    for word in kata:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    kata = "".join(new_words)
    return kata

In [5]:
data = pd.read_excel('cleaned_dataset.xlsx')
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,id,tweet,isp,score
0,0,0,0,0,1.374183e+18,"<product_name> min cek dm , nanyak status kone...",biznet,0
1,1,1,1,1,1.374166e+18,"<product_name> udah min , hbs tweet trs",biznet,1
2,2,2,2,2,1.374153e+18,min cek koneksi internet <product_name>,biznet,0
3,3,3,3,3,1.374139e+18,<user_mention> <user_mention> dm mimin id pela...,biznet,0
4,4,4,4,4,1.374080e+18,<user_mention> <user_mention> dm mimin id kamu...,biznet,0


In [6]:
# remove stopwords
tweets = [clean_text(remove_stopwords_id(str(tweet))) for tweet in data['tweet']]
data['tweet'] = tweets
data.head(10) # data for next process

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,id,tweet,isp,score
0,0,0,0,0,1.374183e+18,"<product_name> min cek dm , nanyak status kone...",biznet,0
1,1,1,1,1,1.374166e+18,"<product_name> udah min , hbs tweet trs",biznet,1
2,2,2,2,2,1.374153e+18,min cek koneksi internet <product_name>,biznet,0
3,3,3,3,3,1.374139e+18,<user_mention> <user_mention> dm mimin id pela...,biznet,0
4,4,4,4,4,1.374080e+18,<user_mention> <user_mention> dm mimin id kamu...,biznet,0
5,5,5,5,5,1.374011e+18,<product_name> perumahan citra garden 1 blok g3,biznet,0
6,6,6,6,6,1.374009e+18,<user_mention> <user_mention> bantu mimin info...,biznet,0
7,7,7,7,7,1.373996e+18,<product_name> .. perumahan citra garden 1 ?,biznet,0
8,8,8,8,8,1.373991e+18,<product_name> halo mau tanya kalo utk pemasan...,biznet,0
9,9,9,9,9,1.373945e+18,"<product_name> <user_mention> gak , rekomendas...",biznet,0


In [7]:
#tfidf
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features=1000, ngram_range=(1, 2))

In [8]:
tweets = data['tweet'].to_list()
vect.fit(tweets)

# sparse matrix
X = vect.transform(tweets)

# transform to an array
my_array = X.toarray()
print(my_array)

#transofrm back to a dataframe, assign column values
X_df = pd.DataFrame(my_array, columns=vect.get_feature_names())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
X_df['score'] = data['score']

In [10]:
# training: 70%-80% of the whole data
from sklearn.model_selection import train_test_split
y = X_df['score']
X = X_df.drop('score', axis=1)

In [12]:
# Non-linear SVM
from sklearn.svm import SVC
svm = SVC()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)

In [14]:
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.7816091954022989

In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix

print("training accuracy: {0:.2f}%".format(svm.score(X_train, y_train)*100))
print("test accuracy: {0:.2f}%".format(svm.score(X_test, y_test)*100))

y_predicted = svm.predict(X_test)
print("test accuracy_score: {0:.2f}%".format(accuracy_score(y_test, y_predicted)*100))
print(confusion_matrix(y_test, y_predicted)/len(y_test))

training accuracy: 94.54%
test accuracy: 78.16%
test accuracy_score: 78.16%
[[0.26436782 0.02298851 0.04597701]
 [0.01724138 0.31609195 0.01149425]
 [0.06321839 0.05747126 0.20114943]]
